In [1]:
# Install dependencies
%pip install -e '.[kafka-python]'

# Import the reliable topic creation functions
from diaspora_event_sdk.sdk.kafka_client import (
    reliable_client_creation,
    KafkaProducer,
    KafkaConsumer,
)
from diaspora_event_sdk import Client as GlobusClient
import json
import os
import uuid
import time
from datetime import datetime

# Initialize client
# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

Obtaining file:///Users/haochen/Documents/diaspora-event-sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diaspora-event-sdk (pyproject.toml) ... done
  Created wheel for diaspora-event-sdk: filename=diaspora_event_sdk-0.4.2-0.editable-py3-none-any.whl size=8267 sha256=090ca39acbdd7af66f481c3d0f5792d02b000e4e24c21d23fb72a472cf3a7edc
  Stored in directory: /private/var/folders/bb/3zwszgbj55d40r9zfwcpsgdc0000gn/T/pip-ephem-wheel-cache-i226f9ko/wheels/5a/e5/35/3821b881ee64c08f315d3f96a9e83b5af40bacde32e4823f2c
Successfully built diaspora-event-sdk
  Attempting uninstall: diaspora-event-sdk
    Found existing installation: diaspora-event-sdk 0.4.2
    Uninstalling diaspora-event-sdk-0.4.2:
      Successfully uninstalled diaspora-event-sdk-0.4.2
Note: you may need to restart the kernel to use updated packag

In [2]:
# Run reliable_client_creation to set up client and test basic flow
topic_name = reliable_client_creation()
print(f"Successfully completed reliable_client_creation: {topic_name}")

Successfully completed reliable_client_creation: ns-ab12747bee03.topic-3b987


In [3]:
# Function to randomly create a topic, produce, and consume messages
def random_topic_produce_consume():
    """Create a random topic, produce 3 messages, and consume them."""
    topic_name = f"topic-{str(uuid.uuid4())[:5]}"
    kafka_topic = f"{c.namespace}.{topic_name}"

    print(f"\n{'=' * 60}")
    print(f"Creating topic: {topic_name}")
    print(f"Kafka topic: {kafka_topic}")
    print(f"{'=' * 60}")

    # Create topic
    topic_result = c.create_topic(topic_name)
    print(f"Topic creation: {json.dumps(topic_result, indent=2, default=str)}")

    if topic_result.get("status") != "success":
        print(f"Failed to create topic: {topic_result}")
        return None

    time.sleep(3)  # Wait for topic to be ready

    # Produce messages
    print(f"\nProducing 3 messages to {kafka_topic}...")
    producer = KafkaProducer(kafka_topic)
    for i in range(3):
        message = {
            "message_id": i + 1,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "content": f"Message {i + 1} from random_topic_produce_consume",
        }
        future = producer.send(kafka_topic, message)
        result = future.get(timeout=30)
        print(f"  Produced message {i + 1}: offset={result.offset}")
    producer.close()

    time.sleep(2)  # Wait before consuming

    # Consume messages
    print(f"\nConsuming messages from {kafka_topic}...")
    consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
    messages = consumer.poll(timeout_ms=10000)
    consumed_count = 0
    for tp, msgs in messages.items():
        for message in msgs:
            data = json.loads(message.value.decode("utf-8"))
            consumed_count += 1
            print(f"  Consumed message {consumed_count}: {data}")
    consumer.close()
    print(f"\nTotal messages consumed: {consumed_count}")

    # Delete topic
    print(f"\nDeleting topic {topic_name}...")
    delete_result = c.delete_topic(topic_name)
    print(f"Topic deletion: {json.dumps(delete_result, indent=2, default=str)}")

    return kafka_topic


# Call the function three times
for i in range(3):
    print(f"\n\n{'#' * 60}")
    print(f"# Iteration {i + 1} of 3")
    print(f"{'#' * 60}")
    result = random_topic_produce_consume()
    if result:
        print(f"✓ Successfully completed iteration {i + 1}: {result}")
    else:
        print(f"✗ Failed iteration {i + 1}")
    if i < 2:  # Don't sleep after last iteration
        time.sleep(2)



############################################################
# Iteration 1 of 3
############################################################

Creating topic: topic-b7f6c
Kafka topic: ns-ab12747bee03.topic-b7f6c
Topic creation: {
  "status": "success",
  "message": "Topic topic-b7f6c created in ns-ab12747bee03",
  "topics": [
    "topic-b7f6c"
  ]
}

Producing 3 messages to ns-ab12747bee03.topic-b7f6c...
  Produced message 1: offset=0
  Produced message 2: offset=1
  Produced message 3: offset=2

Consuming messages from ns-ab12747bee03.topic-b7f6c...
  Consumed message 1: {'message_id': 1, 'timestamp': '2026-01-08 10:30:54', 'content': 'Message 1 from random_topic_produce_consume'}
  Consumed message 2: {'message_id': 2, 'timestamp': '2026-01-08 10:30:54', 'content': 'Message 2 from random_topic_produce_consume'}
  Consumed message 3: {'message_id': 3, 'timestamp': '2026-01-08 10:30:54', 'content': 'Message 3 from random_topic_produce_consume'}

Total messages consumed: 3

Deleting to

In [4]:
# Cleanup: Delete user at the end
print(f"\n{'=' * 60}")
print("Cleaning up: Deleting user...")
print(f"{'=' * 60}")
delete_user_result = c.delete_user()
print(f"User deletion: {json.dumps(delete_user_result, indent=2, default=str)}")
if delete_user_result.get("status") == "success":
    print("✓ User successfully deleted")
else:
    print(f"✗ Failed to delete user: {delete_user_result}")



Cleaning up: Deleting user...
User deletion: {
  "status": "success",
  "message": "User e2a8169b-feef-4d56-8eba-ab12747bee03 deleted"
}
✓ User successfully deleted
